In [4]:

%pip install elasticsearch


[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [6]:
%pip install python-docx

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [7]:
from docx import Document

def clean_word_remove_footer(input_path, output_path):
    # Mở tài liệu Word
    doc = Document(input_path)
    
    for section in doc.sections:
        # Xóa nội dung trong footer
        for paragraph in section.footer.paragraphs:
            paragraph.text = ""  # Làm trống nội dung
        
        # Xóa nội dung trong header (nếu cần)
        for paragraph in section.header.paragraphs:
            paragraph.text = ""  # Làm trống nội dung
    
    # Lưu tài liệu đã làm sạch
    doc.save(output_path)
    print(f"Footer và Header đã được làm trống. Tệp mới lưu tại: {output_path}")

# Sử dụng
clean_word_remove_footer("QUỐC HỘI.docx", "52.2014.QH13_clean.docx")


PackageNotFoundError: Package not found at 'QUỐC HỘI.docx'

In [8]:
from docx import Document

def clean_word_based_on_keywords(input_path, output_path, keywords_to_remove_start, keywords_to_remove_end):
    """
    Xóa các phần đầu và cuối tài liệu dựa trên từ khóa xác định.
    
    input_path: Đường dẫn file gốc (.docx)
    output_path: Đường dẫn file kết quả (.docx)
    keywords_to_remove_start: Danh sách từ khóa để xác định phần đầu không liên quan.
    keywords_to_remove_end: Danh sách từ khóa để xác định phần cuối không liên quan.
    """
    doc = Document(input_path)
    paragraphs = doc.paragraphs
    new_doc = Document()

    # Tìm và loại bỏ các phần đầu không liên quan
    start_index = 0
    for i, paragraph in enumerate(paragraphs):
        if any(keyword in paragraph.text for keyword in keywords_to_remove_start):
            start_index = i + 1  # Bắt đầu từ đoạn sau phần đầu không liên quan
        else:
            break

    # Tìm và loại bỏ các phần cuối không liên quan
    end_index = len(paragraphs)
    for i, paragraph in enumerate(reversed(paragraphs)):
        if any(keyword in paragraph.text for keyword in keywords_to_remove_end):
            end_index = len(paragraphs) - i - 1  # Dừng lại ở đoạn trước phần cuối không liên quan
        else:
            break

    # Thêm các đoạn văn còn lại vào tài liệu mới
    for i, paragraph in enumerate(paragraphs[start_index:end_index]):
        new_doc.add_paragraph(paragraph.text)

    # Lưu tài liệu đã làm sạch
    new_doc.save(output_path)
    print(f"Đã làm sạch tài liệu. Tệp lưu tại: {output_path}")

# Sử dụng:
keywords_to_remove_start = ["Cộng hòa xã hội chủ nghĩa Việt Nam", "Quốc hội", "Chủ tịch"]
keywords_to_remove_end = ["Chữ ký", "Phê duyệt", "Chủ tịch"]

clean_word_based_on_keywords(
    "QUỐC HỘI.docx",
    "52.2014.QH13_clean.docx",
    keywords_to_remove_start=keywords_to_remove_start,
    keywords_to_remove_end=keywords_to_remove_end
)

PackageNotFoundError: Package not found at 'QUỐC HỘI.docx'

In [10]:
from docx import Document
def read_docx(file_path):
    # Đọc nội dung từ file .docx
    doc = Document(file_path)
    text = "\n".join(paragraph.text for paragraph in doc.paragraphs)
    return text

In [11]:
def roman_to_int(roman):
    roman_numerals = {
        "I": 1, "II": 2, "III": 3, "IV": 4, "V": 5, "VI": 6, "VII": 7, "VIII": 8, 
        "IX": 9, "X": 10
    }
    return roman_numerals.get(roman.upper(), None)  # Chuyển chữ hoa và trả về số nếu có

def normalize_chapter_number(chapter_number):
    # Nếu chương là số La Mã, chuyển sang số hệ 10
    try:
        # Nếu là số bình thường (hệ thập phân), không làm gì
        int(chapter_number)
        return chapter_number
    except ValueError:
        # Nếu là số La Mã, chuyển nó thành số hệ 10
        return roman_to_int(chapter_number)  # Trả về số hệ 10 nếu có

In [12]:
import re

def chunk_legal_text(text):
    chunks = []
    current_metadata = {"chuong": None, "muc": None, "dieu": None}

    # Tách văn bản thành từng dòng để dễ xử lý
    lines = text.splitlines()

    i = 0
    while i < len(lines):
        line = lines[i]
        # Bỏ qua các dòng trống
        if not line.strip():
            i += 1
            continue

        # Nhận diện CHƯƠNG
        if re.match(r"^CHƯƠNG ", line):
            match = re.match(r"^CHƯƠNG\s*(\S+)(.*)", line.strip())
            if match:
                chapter_number = match.group(1)
                normalized_chapter_number = normalize_chapter_number(chapter_number)
                current_metadata["chuong"] = f"chương {normalized_chapter_number}"
            
            current_metadata["muc"] = None  # Reset mục

            #Lấy tên chương trên dòng tiếp theo
            i += 1
            if i < len(lines) and lines[i].strip():
                current_metadata["chuong"] += f" {lines[i].strip()}"
            
        
        # Nhận diện Mục
        elif re.match(r"^Mục \d+", line):
            current_metadata["muc"] = line.strip()
            # Lấy tiêu đề của mục (dòng tiếp theo nếu có)
            i += 1
            if i < len(lines) and lines[i].strip():
                current_metadata["muc"] += f" {lines[i].strip()}"
        
        # Nhận diện Điều
        elif re.match(r"^Điều \d+\.", line):
            match_ = re.match(r"^(Điều \d+)\.\s*(.*)", line)
            if match_:
                current_metadata["dieu"] = match_.group(1)  # Lưu "Điều X."
                title_of_dieu = match_.group(2).strip()  # Lưu tên điều (nếu có)
            content_lines = []
            i += 1

            while i < len(lines):
                next_line = lines[i]
                if re.match(r"^(CHƯƠNG |Mục \d+|Điều \d+\.|\\d+\. )", next_line):
                    break
                content_lines.append(next_line.strip())
                i += 1

            # Phân loại điều có một khoản hoặc nhiều khoản
            if content_lines and any(re.match(r"^\d+\. ", line) for line in content_lines):
                # Điều có nhiều khoản
                current_chunk = None
                for idx, content_line in enumerate(content_lines):
                    if re.match(r"^\d+\. ", content_line):  # Bắt đầu khoản mới
                        # Gắn tên điều vào khoản đầu tiên
                        if idx == 0 and title_of_dieu:
                            content_line = f"{title_of_dieu} {content_line}"

                        # Tìm reference trong nội dung khoản
                        references = re.findall(r"Điều \d+", content_line)
                        chunks.append({
                            "chuong": current_metadata["chuong"],
                            "muc": current_metadata["muc"],
                            "dieu": current_metadata["dieu"],
                            "content": content_line.strip(),
                            "reference": references or None,
                        })
                    else:
                        # Xử lý khoản không có đánh số
                        if current_chunk is None:
                            current_chunk = content_line.strip()
                        else:
                            current_chunk += " " + content_line.strip()
                # Nếu có khoản không đánh số thì gắn vào chunks
                if current_chunk:
                    references = re.findall(r"Điều \d+", current_chunk)
                    chunks.append({
                        "chuong": current_metadata["chuong"],
                        "muc": current_metadata["muc"],
                        "dieu": current_metadata["dieu"],
                        "content": current_chunk.strip(),
                        "reference": references or None,
                    })
            else:
                # Điều chỉ có một khoản (nội dung không bắt đầu bằng số)
                body_of_dieu = " ".join(content_lines).strip()
                references = re.findall(r"Điều \d+", body_of_dieu)
                chunks.append({
                    "chuong": current_metadata["chuong"],
                    "muc": current_metadata["muc"],
                    "dieu": current_metadata["dieu"],
                    "content": f"{title_of_dieu} {body_of_dieu}".strip(),
                    "reference": references or None,
                })
        else: 
            i += 1

    return chunks


text = read_docx("data/52.2014.QH13_clean.docx")
text = text.replace("\xa0", " ")
chunks = chunk_legal_text(text)

for chunk in chunks:
    print(chunk)
    print("-" * 40)

{'chuong': 'chương 1 NHỮNG QUY ĐỊNH CHUNG', 'muc': None, 'dieu': 'Điều 1', 'content': 'Phạm vi điều chỉnh Luật này quy định chế độ hôn nhân và gia đình; chuẩn mực pháp lý cho cách ứng xử giữa các thành viên gia đình; trách nhiệm của cá nhân, tổ chức, Nhà nước và xã hội trong việc xây dựng, củng cố chế độ hôn nhân và gia đình.', 'reference': None}
----------------------------------------
{'chuong': 'chương 1 NHỮNG QUY ĐỊNH CHUNG', 'muc': None, 'dieu': 'Điều 2', 'content': 'Những nguyên tắc cơ bản của chế độ hôn nhân và gia đình 1. Hôn nhân tự nguyện, tiến bộ, một vợ một chồng, vợ chồng bình đẳng.', 'reference': None}
----------------------------------------
{'chuong': 'chương 1 NHỮNG QUY ĐỊNH CHUNG', 'muc': None, 'dieu': 'Điều 2', 'content': '2. Hôn nhân giữa công dân Việt Nam thuộc các dân tộc, tôn giáo, giữa người theo tôn giáo với người không theo tôn giáo, giữa người có tín ngưỡng với người không có tín ngưỡng, giữa công dân Việt Nam với người nước ngoài được tôn trọng và được pháp 

In [5]:
%pip install -U langchain-community

^C
Note: you may need to restart the kernel to use updated packages.


   ---------------------------------------- 0.0/2.5 MB ? eta -:--:--
   ------------ --------------------------- 0.8/2.5 MB 8.5 MB/s eta 0:00:01
   ---------------------------------------- 2.5/2.5 MB 9.6 MB/s eta 0:00:00
   ---------------------------------------- 0.0/1.0 MB ? eta -:--:--
   ---------------------------------------- 1.0/1.0 MB 9.6 MB/s eta 0:00:00
  Attempting uninstall: langchain-core
    Found existing installation: langchain-core 0.3.6
    Uninstalling langchain-core-0.3.6:
      Successfully uninstalled langchain-core-0.3.6
  Attempting uninstall: langchain-text-splitters
    Found existing installation: langchain-text-splitters 0.3.0
    Uninstalling langchain-text-splitters-0.3.0:
      Successfully uninstalled langchain-text-splitters-0.3.0
  Attempting uninstall: langchain
    Found existing installation: langchain 0.3.1
    Uninstalling langchain-0.3.1:
      Successfully uninstalled langchain-0.3.1
  Attempting uninstall: langchain-community
    Found existing


[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [13]:
from langchain.vectorstores import ElasticVectorSearch
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.schema import Document
from uuid import uuid4
from elasticsearch import Elasticsearch

In [14]:
# Kết nối với Elasticsearch
from elasticsearch import Elasticsearch
es = Elasticsearch("http://localhost:9200")

In [13]:
pip install tf-keras


   ---------------------------------------- 0.0/1.7 MB ? eta -:--:--
   ------------ --------------------------- 0.5/1.7 MB 4.2 MB/s eta 0:00:01
   ---------------------------------------- 1.7/1.7 MB 7.8 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [20]:
# Tạo Embedding model từ LangChain
embedding_model = HuggingFaceEmbeddings(model_name="sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2")


In [21]:
def normalize_document(document):
    normalized_document = {}
    for key, value in document.items():
        if isinstance(value, str):  # Nếu giá trị là chuỗi, chuyển thành chữ thường
            normalized_document[key] = value.lower()
        elif isinstance(value, list):  # Nếu giá trị là danh sách, chuyển các chuỗi trong danh sách thành chữ thường
            normalized_document[key] = [v.lower() if isinstance(v, str) else v for v in value]
        else:
            normalized_document[key] = value  # Giữ nguyên giá trị không phải chuỗi
    return normalized_document

In [22]:
# Hàm tạo index trong Elasticsearch nếu chưa tồn tại
def create_index_if_not_exists(index_name):
    if not es.indices.exists(index=index_name):
        es.indices.create(index=index_name, body={
            "mappings": {
                "properties": {
                    "embedding_vector": {
                        "type": "dense_vector",
                        "dims": 384  # Số chiều của vector embedding
                    },
                    "chuong": {"type": "text"},
                    "muc": {"type": "text"},
                    "dieu": {"type": "text"},
                    "content": {"type": "text"},
                    "reference": {"type": "text"}
                }
            }
        })

# Hàm lưu dữ liệu lên Elasticsearch
def upload_to_elasticsearch(chunks, index_name):
    for chunk in chunks:
        # Tạo embedding vector cho content
        embedding_vector = embedding_model.embed_query(chunk["content"])
        
        # Dữ liệu để lưu
        document = {
            "embedding_vector": embedding_vector,
            "chuong": chunk["chuong"],
            "muc": chunk["muc"],
            "dieu": chunk["dieu"],
            "content": chunk["content"],
            "reference": chunk["reference"]
        }

        document = normalize_document(document)
        
        # Lưu tài liệu lên Elasticsearch
        es.index(index=index_name, id=str(uuid4()), body=document)

In [23]:
index_name = "legal_documents"  # Tên index
create_index_if_not_exists(index_name)
upload_to_elasticsearch(chunks, index_name)

In [2]:
from elasticsearch import Elasticsearch

# Kết nối Elasticsearch
es = Elasticsearch("http://localhost:9200")

# Tên chỉ mục cần xóa
INDEX_NAME = "legal_documents"

# Kiểm tra và xóa chỉ mục
if es.indices.exists(index=INDEX_NAME):
    es.indices.delete(index=INDEX_NAME)
    print(f"Chỉ mục '{INDEX_NAME}' đã bị xóa thành công.")
else:
    print(f"Chỉ mục '{INDEX_NAME}' không tồn tại.")

Chỉ mục 'legal_documents' đã bị xóa thành công.


In [24]:
from elasticsearch import Elasticsearch
import os

# Khởi tạo Elasticsearch client
es_host = os.getenv("ELASTICSEARCH_HOST", "http://localhost:9200")
es = Elasticsearch(es_host)

# Tên chỉ mục
es_index = "legal_documents"

def print_elasticsearch_data(index_name: str):
    """
    Lấy và in dữ liệu từ Elasticsearch.
    :param index_name: Tên chỉ mục cần truy vấn.
    :param size: Số lượng bản ghi cần lấy.
    """
    try:
        # Truy vấn dữ liệu từ Elasticsearch
        response = es.search(index=index_name, body={"query": {"match_all": {}}})

        # In dữ liệu
        for idx, hit in enumerate(response["hits"]["hits"], start=1):
            source = hit["_source"]
            print(f"Document {idx}:")
            print(f"  Chuong: {source.get('chuong')}")
            print(f"  Muc: {source.get('muc')}")
            print(f"  Dieu: {source.get('dieu')}")
            print(f"  Content: {source.get('content')}")
            print(f"  Reference: {source.get('reference')}")
            print("-" * 40)

    except Exception as e:
        print(f"Error fetching data from Elasticsearch: {e}")

# Gọi hàm để in dữ liệu
print_elasticsearch_data(index_name=es_index)


Document 1:
  Chuong: chương 1 những quy định chung
  Muc: None
  Dieu: điều 1
  Content: phạm vi điều chỉnh luật này quy định chế độ hôn nhân và gia đình; chuẩn mực pháp lý cho cách ứng xử giữa các thành viên gia đình; trách nhiệm của cá nhân, tổ chức, nhà nước và xã hội trong việc xây dựng, củng cố chế độ hôn nhân và gia đình.
  Reference: None
----------------------------------------
Document 2:
  Chuong: chương 1 những quy định chung
  Muc: None
  Dieu: điều 2
  Content: những nguyên tắc cơ bản của chế độ hôn nhân và gia đình 1. hôn nhân tự nguyện, tiến bộ, một vợ một chồng, vợ chồng bình đẳng.
  Reference: None
----------------------------------------
Document 3:
  Chuong: chương 1 những quy định chung
  Muc: None
  Dieu: điều 2
  Content: 2. hôn nhân giữa công dân việt nam thuộc các dân tộc, tôn giáo, giữa người theo tôn giáo với người không theo tôn giáo, giữa người có tín ngưỡng với người không có tín ngưỡng, giữa công dân việt nam với người nước ngoài được tôn trọng và được ph

In [25]:
# Lấy dữ liệu từ Elasticsearch
response = es.search(index=es_index, body={"query": {"match_all": {}}})

# In ra các bản ghi
for hit in response["hits"]["hits"]:
    print(hit["_source"])  # Kiểm tra xem các trường có đúng với mong đợi không


{'embedding_vector': [0.013613025657832623, 0.14445756375789642, -0.09838631004095078, 0.04298406094312668, 0.11847544461488724, 0.33136385679244995, -0.17974801361560822, -0.14098510146141052, 0.0007987211574800313, 0.2186780422925949, 0.2844817042350769, 0.1495162546634674, -0.13882596790790558, -0.06323414295911789, 0.061035722494125366, 0.06443951278924942, -0.15390807390213013, 0.0943002849817276, -0.14807073771953583, 0.24036988615989685, 0.19323450326919556, 0.018813995644450188, -0.24386344850063324, -0.02852741628885269, -0.0178607739508152, -0.12377216666936874, 0.02095933072268963, -0.18379825353622437, 0.13290086388587952, -0.06376378238201141, -0.09282774478197098, 0.0946631208062172, 0.12925784289836884, 0.10988768190145493, -0.17752325534820557, -0.30715224146842957, 0.32254931330680847, -0.17750990390777588, 0.1960240751504898, 0.20919030904769897, 0.15437614917755127, -0.014031133614480495, 0.12141343951225281, -0.15250399708747864, -0.12630398571491241, 0.115748919546

In [7]:
import re

result = "Điều 1"
result_lower = result.lower()

# Use regex to match "điều" or "chương" followed by numbers as a full match
matches = re.findall(r'\b(?:điều|chương)\s*\d+\b', result_lower)

# Join the matches into a string or process them further
filtered_result = ', '.join(matches)
print(filtered_result)


điều 1
